---

In [1]:
source ~/.bashrc

---

第一步：确认你的环境前提

agent-browser 的现实前提有三点，缺一不可。

第一，macOS 本身没有问题，Intel 或 Apple Silicon 都可以。
第二，需要 **Node.js（用于 Playwright）**。
第三，需要 **Rust（用于 agent-browser CLI）**。

你可以先快速确认一下：

In [2]:
node -v
npm -v
rustc --version
cargo --version

v20.19.4
10.8.2
rustc 1.58.1 (db9d1b20b 2022-01-20)
cargo 1.58.0 (f01b232bc 2022-01-19)


如果 Node 没装，建议直接用：

In [ ]:
brew install node

In [ ]:
pnpm setup
pnpm config get global-dir
mkdir -p ~/.pnpm-global
pnpm config set global-dir ~/.pnpm-global
source ~/.zshrc

如果 Rust 没装：

In [ ]:
curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh

In [ ]:
rustup update stable

或者：

In [ ]:
curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y --profile minimal

In [ ]:
rustup update stable

装完记得重新开一个 shell，确保 `cargo` 在 PATH 里。

---

第二步：安装 Playwright（这是最容易被忽略的一步）

agent-browser **不是直接控制浏览器**，它底层仍然依赖 Playwright 的 browser binaries。

在一个干净目录里执行一次即可：

In [ ]:
npx playwright install

或者：

In [ ]:
npm install -g playwright@latest

In [1]:
playwright --version

Version 1.58.2


In [ ]:
playwright install

这一步会下载 Chromium / Firefox / WebKit。
macOS 上第一次运行浏览器时，系统可能会弹出『安全性与隐私』提示，允许即可。

如果这一步没做，后面 agent-browser 会『能启动，但打不开页面』。

---

第三步：安装 agent-browser CLI

现在开始真正的 agent-browser。

官方仓库是：

[https://github.com/vercel-labs/agent-browser](https://github.com/vercel-labs/agent-browser)

最直接、最稳妥的方式是 **从源码安装**：

In [ ]:
git clone https://github.com/vercel-labs/agent-browser.git
cd agent-browser
pnpm install
pnpm build
pnpm build:native   # Requires Rust (https://rustup.rs)
pnpm link --global  # Makes agent-browser available globally
agent-browser install

安装成功后，你应该能看到：

In [1]:
agent-browser --help


agent-browser - fast browser automation CLI for AI agents

Usage: agent-browser <command> [args] [options]

Core Commands:
  open <url>                 Navigate to URL
  click <sel>                Click element (or @ref)
  dblclick <sel>             Double-click element
  type <sel> <text>          Type into element
  fill <sel> <text>          Clear and fill
  press <key>                Press key (Enter, Tab, Control+a)
  hover <sel>                Hover element
  focus <sel>                Focus element
  check <sel>                Check checkbox
  uncheck <sel>              Uncheck checkbox
  select <sel> <val...>      Select dropdown option
  drag <src> <dst>           Drag and drop
  upload <sel> <files...>    Upload files
  download <sel> <path>      Download file by clicking element
  scroll <dir> [px]          Scroll (up/down/left/right)
  scrollintoview <sel>       Scroll element into view
  wait <sel|ms>              Wait for element or time
  screenshot [path]          Take

如果提示 command not found，检查 `$HOME/.cargo/bin` 是否在 PATH 中。

---

第四步：做一次最小可行测试（不接 LLM）

不要一上来就连 LLM，先验证三件事：

1. Playwright 能起浏览器
2. agent-browser 能读 accessibility tree
3. snapshot 能正常输出

按顺序来：

In [2]:
agent-browser open https://miro.com/login/ --headed

✓ Sign in | Miro | The Visual Workspace for Innovation
  https://miro.com/login/


你应该能看到浏览器窗口被打开。

接着：

In [3]:
agent-browser snapshot

- document:
  - button "Skip to main content" [ref=e1]
  - banner:
    - link "Go to Miro main page" [ref=e2]:
      - /url: /index/
      - text: Miro Logo
    - button "Language" [ref=e3]: en
    - text: "Current language is: English"
    - link "Sign up" [ref=e4]:
      - /url: https://miro.com/signup/
  - main:
    - heading "Sign in to Miro" [ref=e5] [level=1]
    - group "Sign-in options":
      - button "SSO" [ref=e6]
      - button "Google" [ref=e7]
      - button "Microsoft 365" [ref=e8]
      - button "Show more sign-in options" [ref=e9]
    - text: Email
    - textbox "Email" [ref=e10]:
      - /placeholder: Enter your email address
    - text: Password
    - textbox "Password" [ref=e11]:
      - /placeholder: Enter your password
    - link "Forgot password?" [ref=e12]:
      - /url: https://miro.com/recover/
    - checkbox "Remember me" [ref=e13] [checked]
    - paragraph:
      - paragraph: Remember me
    - button "Continue with email" [ref=e14]
    - button "Sign in to a

正常情况下，会输出类似：

```
@e1 heading "Example Domain"
@e2 paragraph "This domain is for use in illustrative examples..."
@e3 link "More information..."
```

只要这一步成功，说明你的 **macOS + Playwright + agent-browser 环境是健康的**。

---

第五步：实际操作页面（验证 ref-id 模型）

继续在 example.com 上试：

In [ ]:
agent-browser click @e22
agent-browser snapshot

你会看到页面发生变化，ref-id 全部重新分配。
这一步非常关键，它验证了你对 agent-browser 核心原则的理解：

> ref-id 只在当前 snapshot 有意义

In [ ]:
agent-browser click @e4

In [ ]:
agent-browser fill @e6 "andy@dtype.info"

---

第六步：测试一个真实复杂页面（推荐）

不要立刻上 Miro、Google 这种登录页，先选一个『复杂但不反爬』的页面，比如：

In [ ]:
agent-browser open https://news.ycombinator.com
agent-browser snapshot

你会看到大量：

```
@e1 link "Hacker News"
@e2 link "new"
@e3 link "past"
...
```

试着：

In [ ]:
agent-browser click @e2
agent-browser snapshot

如果这一步稳定，说明 accessibility tree + ref-id 在真实页面上是可用的。

---

第七步：macOS 上几个非常常见的坑

这里是我强烈建议你提前注意的：

1. **中文输入法干扰**
   在 fill textbox 时，尽量用英文输入法，否则某些页面会出现奇怪的 key event。

2. **系统权限**
   第一次运行 Playwright 控制的浏览器时，macOS 可能阻止『自动化控制』。
   去：
   系统设置 → 隐私与安全性 → 辅助功能 / 自动化
   放行你的终端（iTerm / Terminal）。

3. **页面一变就 snapshot**
   不 snapshot 就继续 click / fill，是 agent-browser 使用中最常见的逻辑错误。

---

第八步：准备接 LLM（但暂时别急）

在 macOS 上，推荐的下一步架构是：

* agent-browser 作为 **纯执行层**
* 你自己写一个 **controller（Rust / Python / Node 都行）**
* LLM 只负责：
  * 读 snapshot 文本
  * 输出下一条命令（严格受限）

现在这个阶段，**先不要上 MCP、不要上多 agent**，先把『单步 → snapshot → 单步』这条链跑顺。

---